In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import sqlite3
from datetime import datetime

sns.set_style('whitegrid')

%matplotlib inline

In [95]:
conn = sqlite3.connect(os.getcwd()+'\data\lending-club-loan-data\database.sqlite')
cursor = conn.cursor()
loans2 = pd.DataFrame(cursor.execute('SELECT * FROM LOAN').fetchall())
loans2.columns = np.array(cursor.execute('PRAGMA table_info(loan)').fetchall())[:,1]
conn.close()

In [96]:
loans2.describe()

,index,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,887383.000000,8.873820e+05,887382.000000,887382.000000,887382.000000,887382.000000,8.873780e+05,887382.00000,887353.000000,887353.000000,...,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,8.171060e+05,21372.000000,21372.000000,21372.000000
mean,152155.671062,3.500184e+07,14755.280082,14741.893147,14702.480039,436.717647,7.502761e+04,18.15702,0.314445,0.694622,...,36552.811389,71.535317,1.389060,2.975482,5887.979740,60.831939,3.206868e+04,0.943945,1.537058,1.977307
std,111089.623731,2.411336e+07,8435.456408,8429.898507,8442.107612,244.186619,6.469823e+04,17.19060,0.862247,0.998447,...,43103.833619,23.022862,1.520129,2.631886,5284.701239,20.013254,3.749833e+04,1.446872,2.723339,2.874067
min,0.000000,7.047300e+04,500.000000,500.000000,0.000000,15.670000,0.000000e+00,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,59769.500000,1.087713e+07,8000.000000,8000.000000,8000.000000,260.710000,4.500000e+04,11.91000,0.000000,0.000000,...,10251.500000,58.600000,0.000000,1.000000,2411.000000,47.700000,1.390000e+04,0.000000,0.000000,0.000000
50%,133718.000000,3.709528e+07,13000.000000,13000.000000,13000.000000,382.550000,6.500000e+04,17.65000,0.000000,0.000000,...,24684.500000,74.900000,1.000000,2.000000,4483.000000,61.900000,2.370000e+04,0.000000,0.000000,2.000000
75%,217438.500000,5.847135e+07,20000.000000,20000.000000,20000.000000,572.600000,9.000000e+04,23.95000,0.000000,1.000000,...,47858.000000,87.600000,2.000000,4.000000,7771.500000,75.200000,3.980000e+04,1.000000,2.000000,3.000000
max,421094.000000,7.354484e+07,35000.000000,35000.000000,35000.000000,1445.460000,9.500000e+06,9999.00000,39.000000,33.000000,...,878459.000000,223.300000,22.000000,43.000000,127305.000000,151.400000,9.999999e+06,17.000000,35.000000,32.000000


In [97]:
loans2.last_pymnt_d.head()

0    Jan-2015
1    Apr-2013
2    Jun-2014
3    Jan-2015
4    Jan-2016
Name: last_pymnt_d, dtype: object

In [98]:
# loan_status will be the target variable. 10 possible classes

for status in np.unique(loans2['loan_status'].astype(str)):
    print(status)
    print(str(len(loans2[loans2['loan_status']==str(status)])))

Charged Off
45248
Current
601781
Default
1219
Does not meet the credit policy. Status:Charged Off
761
Does not meet the credit policy. Status:Fully Paid
1988
Fully Paid
207724
In Grace Period
6253
Issued
8460
Late (16-30 days)
2357
Late (31-120 days)
11591
None
0


In [99]:
# drop columns which have > 400k NaN values?
# may be able to figure out to fill values for next_pymnt_d (250k missing values)

loans2.isnull().sum().sort_values(ascending=False)[:30]

dti_joint                      886874
verification_status_joint      886872
annual_inc_joint               886872
il_util                        868766
mths_since_rcnt_il             866573
all_util                       866011
max_bal_bc                     866011
open_rv_24m                    866011
open_rv_12m                    866011
total_cu_tl                    866011
total_bal_il                   866011
open_il_24m                    866011
open_il_12m                    866011
open_il_6m                     866011
open_acc_6m                    866011
inq_fi                         866011
inq_last_12m                   866011
desc                           761354
mths_since_last_record         750330
mths_since_last_major_derog    665679
mths_since_last_delinq         454314
next_pymnt_d                   252973
tot_cur_bal                     70277
total_rev_hi_lim                70277
tot_coll_amt                    70277
emp_title                       51458
last_pymnt_d

In [100]:
loans2.columns

Index(['index', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_sta

In [175]:
# strange bug not allowing the whole column to be changed, b/c of a single NaN value that wont be removed..

loans2['term'] = loans2[loans2['term']!='nan']['term']
loans2['term'] = loans2['term'].dropna()

import re 

f = lambda x : int(re.findall(r"\D(\d{2})\D",x)[0])

loans2['term'].head().apply(f)

0    36
1    60
2    36
3    36
4    60
Name: term, dtype: int64

In [126]:
np.unique(loans2['term'].astype(str).values)

array([' 36 months', ' 60 months', 'nan'], dtype=object)

In [161]:
# dropna not dropping a single NaN...weird bug

loans2['term'][39783:39789]

39783     36 months
39784     36 months
39785     36 months
39786           NaN
39787     60 months
39788     60 months
Name: term, dtype: object

In [192]:
# example "total loan repayment"

loans2['installment'].head()*loans2['term'].head().apply(f)

0     5863.32
1     3589.80
2     3035.88
3    12215.16
4     4067.40
dtype: float64

In [195]:
# example "total net % return" (over entire life of loan)

((loans2['installment'].head()*loans2['term'].head().apply(f))/loans2['funded_amnt'].head() - 1)*100

0    17.2664
1    43.5920
2    26.4950
3    22.1516
4    35.5800
dtype: float64